# LangGraph (notater) — fra “chains” til graf-baserte agent-systemer

Dette kapittelet forklarer LangGraph på en “bachelor-vennlig” måte:

- hva LangGraph er
- hvorfor graf-modell er nyttig (state, kontrollflyt, retries)
- hvordan det henger sammen med LangChain (LCEL/Runnables)
- hvordan du bygger en enkel agent/RAG-flow som graf


## 1) Hva er LangGraph?

**LangGraph** er et rammeverk for å bygge **stateful** LLM-applikasjoner som en **graf**:

- **noder** = steg (LLM-kall, retrieval, parsing, validering, tool call)
- **kanter** = dataflyt / kontrollflyt (hva skjer etter hva)
- **state** = en delt datastruktur som følger hele kjøringen (som “minne”/kontekst)

Det er spesielt nyttig når du har mer enn én lineær chain, f.eks:

- routing (hvilken vei i flyten?)
- loops (prøv igjen / iterer)
- human-in-the-loop (stopp og be om godkjenning)
- robusthet (retries, checkpoints)


## 2) LangChain vs LangGraph (enkelt forklart)

- **LangChain**: byggesteiner (loaders, splitters, retrievers, prompt templates, output parsers, LCEL/runnables)
- **LangGraph**: *orkestrering som graf* når flyten trenger kontroll og state

Du kan tenke:

- LangChain = “komponenter + lineær pipeline”
- LangGraph = “pipeline som graf med branching/loops og tydelig state”

I praksis bruker du ofte begge:

- LangChain for RAG-komponentene
- LangGraph for agent-loop + routing + policy/guardrails


## 3) Kjernebegreper

### State

En “state” er typisk en dict/dataclass/TypedDict som inneholder:

- `messages` (chat-historikk)
- `question` / `task`
- `retrieved_docs`
- `answer`
- `errors` / `retries`
- `metadata` (kilder, timing, tokenbruk)

### Nodes

En node er en funksjon som:

- tar inn state
- gjør et steg
- returnerer endringer i state

### Edges + router

Edges definerer neste steg.

En **router** er en node som bestemmer hvilken vei du tar videre (f.eks. basert på klassifisering eller validering).

### Checkpointing

LangGraph kan lagre state mellom steg, nyttig for:

- debugging
- “resume”
- revisjon/sporbarhet

### Human-in-the-loop

I noen flows må menneske godkjenne:

- før et tool kalles
- før svar sendes til kunde
- når policy flagger PII eller høy risiko


## 4) Typiske grafer i “kjedelige bransjer” (kundeservice/IT/finans)

### A) Smart ticket routing + RAG

1. ingest ticket
2. PII-maskering
3. klassifiser kategori/prioritet
4. retrieve relevante runbooks/SLA
5. generer svarutkast
6. output parsing (maskinlesbart)
7. policy/validering
8. logg/tracing
9. send til ticketsystem

### B) Agent med verktøy

- node: plan
- node: tool-call (DB, docs, API)
- node: synthesize answer
- router: trenger mer info? eller ferdig?

### C) “Guardrail graph”

- input → PII-detektor → enten maskér eller stopp
- output → policy-check → enten send eller be om menneske


In [ ]:
# LangGraph (kode-mal) — minimal state + nodes + routing
# NB: Dette er en *template* (ikke kjørbar uten langgraph/langchain install).

# from typing import TypedDict, List, Literal
# from langgraph.graph import StateGraph, END
#
# class GraphState(TypedDict, total=False):
#     question: str
#     category: str
#     answer: str
#
# def classify(state: GraphState) -> GraphState:
#     q = state["question"].lower()
#     if "sla" in q or "shipping" in q:
#         return {"category": "logistics"}
#     if "payment" in q or "chargeback" in q:
#         return {"category": "payments"}
#     return {"category": "general"}
#
# def route(state: GraphState) -> Literal["logistics", "payments", "general"]:
#     return state.get("category", "general")
#
# def answer_general(state: GraphState) -> GraphState:
#     return {"answer": "General answer stub"}
#
# def answer_logistics(state: GraphState) -> GraphState:
#     return {"answer": "Logistics answer stub (would use RAG)"}
#
# def answer_payments(state: GraphState) -> GraphState:
#     return {"answer": "Payments answer stub (would use RAG + policy)"}
#
# g = StateGraph(GraphState)
# g.add_node("classify", classify)
# g.add_node("general", answer_general)
# g.add_node("logistics", answer_logistics)
# g.add_node("payments", answer_payments)
#
# g.set_entry_point("classify")
# g.add_conditional_edges("classify", route, {
#     "general": "general",
#     "logistics": "logistics",
#     "payments": "payments",
# })
#
# g.add_edge("general", END)
# g.add_edge("logistics", END)
# g.add_edge("payments", END)
#
# app = g.compile()
# print(app.invoke({"question": "What is our SLA for delayed shipping?"}))


## 5) Hvordan knytte dette til bachelorteksten din

Du kan beskrive LangGraph-delen slik (rapportvennlig):

- LangGraph modellerer applikasjonsflyten som en graf der hvert steg er en node.
- State følger hele kjøringen og gjør løsningen **stateful** og sporbar.
- Routing/loops gjør at systemet kan velge riktig strategi (f.eks. RAG vs direkte svar) og håndtere feil/retries.
- Graf-arkitektur gjør det enklere å plassere sikkerhetskontroller (PII-maskering, policy-check) som eksplisitte noder.

Hvis du vil, kan jeg også lage en *egen* notebook `02-langgraph-rag-agent.ipynb` som viser en komplett “RAG + tool-use + output parsing” graf i én sammenhengende mal.
